In [298]:
import csv
import sys
import xgt
import os
import pandas
from platform import python_version

In [299]:
if os.environ.get('https_proxy'):
 del os.environ['https_proxy']
if os.environ.get('http_proxy'):
 del os.environ['http_proxy']

In [300]:
print (python_version())
conn=xgt.Connection()
conn.server_version

3.7.4


'1.3.0'

In [301]:
print("Job Monitoring")
all_jobs = conn.get_jobs()

#if len(all_jobs) == 0:
#    print("No Jobs!!!")

for j in all_jobs:
    print(j)
    
#Cancel all running jobs (if required)
for job in all_jobs:
    if job.status == 'running':
        conn.cancel_job(job)
        print ("calcelled JobId = {0}",job)
    

Job Monitoring
id:2686, status:completed
id:2724, status:completed
id:2746, status:completed
id:2752, status:completed
id:2780, status:completed
id:2818, status:completed


In [302]:
#If your data is LANL Data then fetch details of lanl data loaded in xgt
print("Nodes/Edges Monitoring LANL")
devices = conn.get_vertex_frame('Devices')
netflow = conn.get_edge_frame('Netflow')
events1v = conn.get_edge_frame('HostEvents')
events2v = conn.get_edge_frame('AuthEvents')

print('Devices (vertices): {:,}'.format(devices.num_vertices))
print('Netflow (edges): {:,}'.format(netflow.num_edges))
print('Host-only event (edges): {:,}'.format(events1v.num_edges))
print('Host-communication event (edges): {:,}'.format(events2v.num_edges))
print('Total number of edges: {:,}'.format(netflow.num_edges + events1v.num_edges + events2v.num_edges))

Nodes/Edges Monitoring LANL
Devices (vertices): 137,812
Netflow (edges): 235,661,328
Host-only event (edges): 18,637,483
Host-communication event (edges): 47,790,045
Total number of edges: 302,088,856


In [303]:
#If your data is generic and not lanl data
print("Nodes/Edges Monitoring NON-LANL")
tables = conn.get_table_frames()
for table in tables:
    print("TableFrame {0} has {1:,} rows".format(table.name, table.num_rows))
    
vertices = conn.get_vertex_frames()
for vertex in vertices:
    print("VertexFrame {0} has {1:,} rows".format(vertex.name, vertex.num_rows))

edges = conn.get_edge_frames()
for edge in edges:
    print("EdgeFrame {0} has {1:,} rows".format(edge.name, edge.num_rows))


Nodes/Edges Monitoring NON-LANL
TableFrame answers has 10,572 rows
VertexFrame Devices has 137,812 rows
EdgeFrame Netflow has 235,661,328 rows
EdgeFrame HostEvents has 18,637,483 rows
EdgeFrame AuthEvents has 47,790,045 rows
EdgeFrame RDPFlow has 9,732 rows
EdgeFrame HijackEvents has 166,889 rows
EdgeFrame PrivEscEvents has 135,214 rows


In [304]:
#Print specific table data
print("Print Specific Table Data")
tableName = "answers"
table = conn.get_table_frame(tableName)
numRows = table.num_rows
schema = table.schema

if numRows == 0:
    print("Zero Rows in Table")
    
#print schema
print("Print Table Schema for table {0} :",tableName)
result = ""
for columns in schema:
    result += "\n"
    colName = columns[0]
    if columns[1] == 'int':
        colType = 'xgt.INT'
    elif columns[1] == 'text':
        colType = 'xgt.TEXT'
    elif columns[1] == 'float':
        colType = 'xgt.FLOAT'
    elif columns[1] == 'datetime':
        colType = 'xgt.DATETIME'
    else:
        colType = 'xgt.UNKNOWN'
    if len(result) > 0:
        result += ", "
    result += "{0}:{1}".format(colName, colType)    
print(result)

#print data
print("Print data for table {0} :",tableName)
data = table.get_data_pandas()
data[0:10]
data

Print Specific Table Data
Print Table Schema for table {0} : answers

, rdp1_src_device:xgt.TEXT
, rdp1_dst_device:xgt.TEXT
, rdp1_epoch_time:xgt.INT
, rdp2_dst_device:xgt.TEXT
, rdp2_epoch_time:xgt.INT
Print data for table {0} : answers


,rdp1_src_device,rdp1_dst_device,rdp1_epoch_time,rdp2_dst_device,rdp2_epoch_time
0,ActiveDirectory,EnterpriseAppServer,7290427,Comp503532,7291140
1,ActiveDirectory,EnterpriseAppServer,7290427,Comp503532,7291140
2,ActiveDirectory,EnterpriseAppServer,7290427,Comp503532,7291140
3,ActiveDirectory,EnterpriseAppServer,7290427,Comp503532,7291140
4,ActiveDirectory,EnterpriseAppServer,7290438,Comp503532,7291140
...,...,...,...,...,...
10567,ActiveDirectory,EnterpriseAppServer,7290427,Comp257274,7292630
10568,ActiveDirectory,EnterpriseAppServer,7290427,Comp257274,7292630
10569,ActiveDirectory,EnterpriseAppServer,7290427,Comp257274,7292630
10570,ActiveDirectory,EnterpriseAppServer,7290427,Comp257274,7292630


In [ ]:
#Generate Graph from Pandas Dataframes
from_pandas_dataframe(df, source, target, edge_attr=None, create_using=None)

In [305]:
#Data Pre-processing
input_file="/nvme_data1/data_2v/wls_day-86_2v.csv"
output_file="/nvme_data9/data_2v/wls_day-86_2v.csv"

print("Pre-processing specific field in input file : {0}",input_file)

LOGHOST = 2
SOURCE = 12
DEST = 14

with open(output_file, 'w') as outfile:
    writer = csv.writer(outfile, quoting=csv.QUOTE_MINIMAL)
    with open(input_file, 'r') as infile:
        reader = csv.reader(infile)
        for row in reader:
            if row[DEST] == '':
                row[DEST] = row[LOGHOST]
            if row[SOURCE] == '':
                row[SOURCE] = row[LOGHOST]
            writer.writerow(row)
            
print("Output file generation complete : {0}",output_file)            

Pre-processing specific field in input file : {0} /nvme_data1/data_2v/wls_day-86_2v.csv
Output file : {0} /nvme_data9/data_2v/wls_day-86_2v.csv


In [309]:
%%bash

#ls -altr

#GET CIC DDOS2019 Dataset in pcap format
#wget "http://205.174.165.80/CICDataset/CICDDoS2019/Dataset/PCAPs/03-11/PCAP-03-11.zip" -O “/nvme_data14/CICDataset/CICDDoS2019/Dataset/PCAPs/PCAP-03-11.zip"
#wget "http://205.174.165.80/CICDataset/CICDDoS2019/Dataset/PCAPs/01-12/PCAP-01-12_0-0249.zip" -O "/nvme_data14/CICDataset/CICDDoS2019/Dataset/PCAPs/PCAP-01-12_0-0249.zip"
#wget "http://205.174.165.80/CICDataset/CICDDoS2019/Dataset/PCAPs/01-12/PCAP-01-12_0250-0499.zip" -O "/nvme_data14/CICDataset/CICDDoS2019/Dataset/PCAPs/PCAP-01-12_0250-0499.zip"
#wget "http://205.174.165.80/CICDataset/CICDDoS2019/Dataset/PCAPs/01-12/PCAP-01-12_0500-0749.zip" -O "/nvme_data14/CICDataset/CICDDoS2019/Dataset/PCAPs/PCAP-01-12_0500-0749.zip"
#wget "http://205.174.165.80/CICDataset/CICDDoS2019/Dataset/PCAPs/01-12/PCAP-01-12_0750-0818.zip" -O "/nvme_data14/CICDataset/CICDDoS2019/Dataset/PCAPs/PCAP-01-12_0750-0818.zip"

#GET CIC DDOS2019 Dataset in csv format
#wget "http://205.174.165.80/CICDataset/CICDDoS2019/Dataset/CSVs/CSV-01-12.zip" -O "/nvme_data14/CICDataset/CICDDoS2019/Dataset/csv/CSV-01-12.zip"
#wget "http://205.174.165.80/CICDataset/CICDDoS2019/Dataset/CSVs/CSV-03-11.zip" -O "/nvme_data14/CICDataset/CICDDoS2019/Dataset/csv/CSV-03-11.zip"

#GET LANL Dataset in csv format
#for i in {1..90}; do wget "https://datasets.trovares.com/LANL/xgt/wls_day-"$i"_2v.csv" -O "/nvme_data1/data_1v/wls_day-"$i"_2v.csv" ; done
#for i in {1..90}; do wget "https://datasets.trovares.com/LANL/xgt/wls_day-"$i"_1v.csv" -O "/nvme_data1/data_2v/wls_day-"$i"_1v.csv" ; done
#for i in {1..90}; do wget "https://datasets.trovares.com/LANL/xgt/nf_day-"$i".csv" -O "/nvme_data1/data_nf/nf_day-"$i".csv" ; done

